<a href="https://colab.research.google.com/github/harshalc108/Project-demo-pipelines/blob/main/Streaming_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install apache_beam[gcp]

In [ ]:
from apache_beam.transforms.window import FixedWindows
from apache_beam.io.gcp.bigquery import WriteToBigQuery
from apache_beam.options.pipeline_options import PipelineOptions, StandardOptions
import argparse
import os
import time
from apache_beam.options.pipeline_options import SetupOptions
from datetime import date
from google.cloud import bigquery
from apache_beam import window
from apache_beam.transforms.trigger import AfterWatermark, AfterProcessingTime, AccumulationMode, AfterCount, Repeatedly

path1='bigquery-demo-335405-d7f1212692d0.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=path1
tab_id='StreamingData.sales_mode'
import apache_beam as beam

parser = argparse.ArgumentParser()
# parser.add_argument('--my-arg')
args, beam_args = parser.parse_known_args()

# Create and set your Pipeline Options.
beam_options = PipelineOptions(
    beam_args,
    runner='DataflowRunner',
    temp_location='gs://demo_data0/temp1999/',
    service_account_email='demo-python@bigquery-demo-335405.iam.gserviceaccount.com',
    region='us-central1',
    project='bigquery-demo-335405',
    stage_location='gs://data_00/stage',
    #save_main_session=True
    )
beam_options.view_as(StandardOptions).streaming=True
#beam_options.view_as(SetupOptions).save_main_session = True 
client = bigquery.Client()

dataset_id = "bigquery-demo-335405.StreamingData"

try:
  client.get_dataset(dataset_id)

except:
  dataset = bigquery.Dataset(dataset_id)  

  dataset.location = "US"
  dataset.description = "dataset"

  dataset_ref = client.create_dataset(dataset)
 
def decoding(e):
  e=e.decode('utf-8')
  return e[:-2]
class AddTimestampDoFn(beam.DoFn):
  def process(self,element):
    yield TimestampedValue(element)
 
input_subscription='projects/bigquery-demo-335405/subscriptions/data_from_file_sub'
def to_json(fields):
    json_str = {"Region":fields[0],
                 "Country": fields[1],
                 "Item_Type": fields[2],
                 "Sales_Channel": fields[3],
                "Order_Priority": fields[4],
                "Order_Date": fields[5],
                "Order_ID": fields[6],
                "Ship_Date": fields[7],
                "Units_Sold": fields[8],
                "Unit_Cost": fields[9],
                "Total_Profit": fields[10]
                 }
    
    return json_str
schema_1='Region:STRING,Country:STRING,Item_Type:STRING,Sales_Channel:STRING,Order_Priority:STRING,Order_Date:STRING,Order_ID:INTEGER,Ship_Date:STRING,Units_Sold:INTEGER,Unit_Cost:FLOAT,Total_Profit:FLOAT'
with beam.Pipeline(options=beam_options) as pipeline:
  data_ingestion=(
    pipeline
    | beam.io.ReadFromPubSub(subscription=input_subscription,timestamp_attribute=None)
    | beam.Map(decoding)
    | beam.Map(lambda x:x.split(','))
    | beam.Map(to_json)
   # | beam.Map(lambda x:beam.window.TimestampedValue(x,time.time()))
   # | beam.ParDo(AddTimestampDoFn())
    | beam.WindowInto(beam.window.FixedWindows(5))
    # | beam.CombinePerKey()
    | beam.io.WriteToBigQuery(
                             tab_id,
                             schema=schema_1,
                             method=                                                                                
                     beam.io.WriteToBigQuery.Method.FILE_LOADS,
                             triggering_frequency=1, 
                             write_disposition=
                     beam.io.BigQueryDisposition.WRITE_APPEND,
                             create_disposition=
                     beam.io.BigQueryDisposition.CREATE_IF_NEEDED
        
    )
)